In [2]:
!pip install tensorflow-gpu

In [3]:
import tensorflow as tf
import keras

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
tf.__version__

'2.5.0'

In [5]:
import struct, os

In [19]:
def load_mnist(path, kind = 'train'):
    labels_path = os.path.join(path, "%s-labels-idx1-ubyte"%kind)
    images_path = os.path.join(path, "%s-images-idx3-ubyte"%kind)
    # label
    with open(labels_path, "rb") as la_path:
        magic, n = struct.unpack(">II", la_path.read(8))
        labels = np.fromfile(la_path, dtype = np.uint8)
    
    # image
    with open(images_path, "rb") as img_path:
        magic, num, rows, cols = struct.unpack(">IIII", img_path.read(16))
        images = np.fromfile(img_path, dtype = np.uint8).reshape(len(labels), 28**2)
        images = ((images/255)-0.5)*2
    
    return images, labels

In [25]:
X_train, y_train = load_mnist("C:/Users/CPB06GameN/Desktop/image_label_ubyte", kind = "train")
X_test, y_test = load_mnist("C:/Users/CPB06GameN/Desktop/image_label_ubyte", kind = "t10k")
print(X_train.shape[0], X_train.shape[1])
print(X_test.shape[0], X_test.shape[1])

60000 784
10000 784


In [26]:
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)
X_train_centered = (X_train-mean_vals)/std_val
X_test_centered = (X_test-mean_vals)/std_val

In [27]:
y_train_onehot = tf.keras.utils.to_categorical(y_train)
y_train_onehot[:3]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)

In [28]:
y_train[:3]

array([5, 0, 4], dtype=uint8)

In [33]:
model = tf.keras.models.Sequential()

model.add(
    tf.keras.layers.Dense(
        units = 50,
        input_dim = X_train_centered.shape[1],
        kernel_initializer = 'glorot_uniform',
        bias_initializer = 'zeros',
        activation = "tanh"
    )
)
model.add(
    tf.keras.layers.Dense(
        units = 50,
        input_dim = 50,
        kernel_initializer = 'glorot_uniform',
        bias_initializer = 'zeros',
        activation = "tanh"
))
model.add(
    tf.keras.layers.Dense(
        units = 10,
        input_dim = 50,
        kernel_initializer = 'glorot_uniform',
        bias_initializer = 'zeros',
        activation = "softmax"
))

In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                39250     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                510       
Total params: 42,310
Trainable params: 42,310
Non-trainable params: 0
_________________________________________________________________


In [36]:
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.0001, decay=1e-7, momentum=0.9)
model.compile(optimizer=sgd_optimizer, loss ="categorical_crossentropy")

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [ ]:
model.fit(X_train_centered, y_train_onehot, batch_size=64, epochs=50, verbose=1,)